# Generating Joy Division Plots for Distributions of contacts across R8 data

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns

#set file path
path = '/Users/sophieayling/Library/CloudStorage/GoogleDrive-sophie2ayling@gmail.com/My Drive/PhD/08_Fieldwork/08_Data/R8_data/stata_input/'
data_wide = pd.read_stata(path+'IVQ_R8_newVars_200cutoff.dta')
path_output= '/Users/sophieayling/Library/CloudStorage/GoogleDrive-sophie2ayling@gmail.com/My Drive/PhD/08_Fieldwork/08_Data/R8_data/plots/final_paper/q3/'
cut_off= "_200cutoff"
# path = '/Users/sophieayling/Library/CloudStorage/GoogleDrive-sophie2ayling@gmail.com/My Drive/PhD/08_Fieldwork/08_Data/R8_data/stata_input/'
#path2 = ''

# data_wide = pd.read_stata(path + 'IVQ_R8_newVars.dta')
data_wide = data_wide[['hhmem_key', 'ses_5', 'ses', 'age_5', 'age', 'gender', 'site_type', 'occ_2',
                       'occupations', 'tot_contacts_work_wk_ld', 'tot_contacts_school_wk_ld', 'tot_contacts_home_wk_ld',
                       'tot_contacts_trans_wk_ld', 'tot_contacts_ent_we_ld',  'nld_all_ages_contacts_week',
                       'ld_all_ages_contacts_week', 'tot_contacts_work_wk_nld', 'tot_contacts_school_wk_nld',
                       'tot_contacts_home_wk_nld', 'tot_contacts_trans_wk_nld', 'tot_contacts_ent_we_nld',
                       'm_tot_contacts_work_wk_nld', 'm_tot_contacts_home_wk_nld', 'm_tot_contacts_trans_wk_nld',
                       'm_tot_contacts_ent_we_nld', 'm_tot_contacts_work_wk_ld', 'm_tot_contacts_home_wk_ld',
                       'm_tot_contacts_trans_wk_ld', 'm_tot_contacts_ent_we_ld', 'm_diff_home_wk_ld',
                       'm_diff_work_wk_ld', 'm_diff_ent_we_ld', 'm_diff_trans_wk_ld']]
grouped_data = data_wide.groupby('hhmem_key').mean()
grouped_cat = data_wide.groupby('hhmem_key').nth(0)


grouped_data['occupation'] = grouped_cat['occupations']
grouped_data['occ_2']= grouped_cat['occ_2']
grouped_data['gender']= grouped_cat['gender']
grouped_data['ses_5']=grouped_cat['ses_5']
grouped_data['age_5']=grouped_cat['age_5']

grouped_data.head()


# rename the ses variables
# ses_numbers = list(grouped_data['ses_5'].unique())
# ses_numbers.sort()
# ses_cats = [f"ses_{1 + i}" for i, val in enumerate(ses_numbers)]
# for idx, val in enumerate(ses_numbers):
#     grouped_data.loc[grouped_data['ses'] == val, 'ses'] = \
#         [ses_cats[idx]] * len(grouped_data.loc[grouped_data['ses'] == val])

columns_to_produce_graphs_for = ['age_5', 'gender', 'occ_2', 'ses_5']
grouped_data = grouped_data.dropna(subset=columns_to_produce_graphs_for)
contacts_columns = ['nld_all_ages_contacts_week', 'ld_all_ages_contacts_week']

# Create places to store the x and y values of each kde
nld_x_vals = []
ld_x_vals = []

nld_y_vals = []
ld_y_vals = []

category_dict = dict()

for cat_col in columns_to_produce_graphs_for:
    unique_cats = grouped_data[cat_col].unique()
    if cat_col == 'age_5':
        unique_cats = ['15-29 yrs', '30-44 yrs', '45-64 yrs', '65+ yrs']
    if cat_col == 'ses_5':
        unique_cats = [1.0, 2.0, 3.0, 4.0, 5.0]
    # unique_cats = [unique_cats[i] for i in sorted(np.unique(unique_cats, return_index=True)[1])]
    for cat in unique_cats:
        print(cat)
        df = grouped_data.loc[grouped_data[cat_col] == cat]
        # Create the kde line for this category
        fig, ax = plt.subplots()
        line = sns.kdeplot(df['nld_all_ages_contacts_week'], label=cat)
        # Get the actual line from the graph
        try:
            line = line.lines[0]
        except IndexError:
            pass
        plt.close(fig)

        fig, ax = plt.subplots()
        line2 = sns.kdeplot(df['ld_all_ages_contacts_week'])

        # Get the actual line from the graph
        try:
            line2 = line2.lines[0]
        except IndexError:
            pass

        # Get the corresponding x and y values so we can use them later
        try:
            x_nld, y_nld = line.get_data()
            x_ld, y_ld = line2.get_data()
        except AttributeError:
            pass

        # Store this information in a dictionary
        category_dict[str(cat) + "__nld"] = [x_nld, y_nld]
        category_dict[str(cat) + "__ld"] = [x_ld, y_ld]

        # Store these x and y values
        nld_x_vals.append(x_nld)
        nld_y_vals.append(y_nld)
        ld_x_vals.append(x_ld)
        ld_y_vals.append(y_ld)
        plt.close()


# iterate over the names to plot the ld and nld variants
y_vals_for_scaling = [0]
for col in category_dict.keys():
    if '__nld' in col:
        x, y = category_dict[col]
        y_vals_for_scaling.append(max(y))

# We now need to calculate the cumulative y_maxs values to offset out curves
y_offset = np.cumsum(y_vals_for_scaling)
# We'll likely need to scale the y values some more so we'll do so here
y_offset *= 2.5
colours = ['r', 'b'] * 19
doubled_offset = []
for i in y_offset:
    for j in range(0, 2):
        doubled_offset.append(i)

new_cat_dict = dict()

for idx, (x, y) in enumerate(category_dict.values()):
    new_cat_dict[list(category_dict.keys())[idx]] = [x, y + doubled_offset[idx]]
plt.figure(figsize=(10, 12))
for idx, (x, y) in enumerate(new_cat_dict.values()):
    # First off plot the line, specifying the colour from colours
    plt.plot(x, y, color=colours[idx], alpha=1)
    # Now we need to get a base for the shading, this will just be the minimum value of the y_values for this curve.
    # For the function plt.fill_between, we need to have the same number of x, y_min and y_max values otherwise the
    # function won't work so let's create the minimum values for this shading
    nld_y_fill_min = [min(y)] * len(x)
    # Now plot the shaded region
    plt.fill_between(x, nld_y_fill_min, y, color=colours[idx], alpha=0.2)

# get unique names for lock down and non-lockdown columns
col_names = [i.split("__")[0] for i in list(category_dict.keys())]
col_names = [col_names[i] for i in sorted(np.unique(col_names, return_index=True)[1])]
print(col_names)
new_names = []
for name in col_names:
    name_to_add = ''
    if name == '5.0':
        name_to_add = 'ses_5'
    elif name == '4.0':
        name_to_add = 'ses_4'
    elif name == '3.0':
        name_to_add = 'ses_3'
    elif name == '2.0':
        name_to_add = 'ses_2'
    elif name == '1.0':
        name_to_add = 'ses_1'
    else:
        name_to_add = name
    new_names.append(name_to_add)
plt.subplots_adjust(left=0.3)
orig_xlim = plt.xlim()
orig_ylim = plt.ylim()

plt.plot([-1000, -1000], [-1000, -1100], label='NLD', color='r', alpha=1)
plt.plot([-1000, -1000], [-1000, -1100], label='LD', color='b', alpha=1)
plt.xlim(orig_xlim)
plt.ylim(orig_ylim)
plt.yticks(y_offset[:-1], labels=new_names, fontsize=7.5)
plt.xlabel('Number of contacts')
plt.legend(loc='upper right', fancybox=True, framealpha=0.5)
plt.title("Distribution of weekday contacts across subgroups \nduring lockdown and non-lockdown", weight='bold')
plt.savefig(path_output+ "stacked_kde"+cut_off+".png", dpi=300, facecolor='w')

TypeError: 'Categorical' with dtype category does not support reduction 'mean'

## doing another one just for occupations according to model categories

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns

#set file path
path = '/Users/sophieayling/Library/CloudStorage/GoogleDrive-sophie2ayling@gmail.com/My Drive/PhD/08_Fieldwork/08_Data/R8_data/stata_input/'
data_wide = pd.read_stata(path+'IVQ_R8_newVars_200cutoff_econ_stat_only.dta')
path_output= '/Users/sophieayling/Library/CloudStorage/GoogleDrive-sophie2ayling@gmail.com/My Drive/PhD/08_Fieldwork/08_Data/R8_data/plots/final_paper/'
cut_off= "_200cutoff"


grouped_data = data_wide.groupby('hhmem_key').mean()
grouped_cat = data_wide.groupby('hhmem_key').nth(0)


grouped_data['economic_status'] = grouped_cat['economic_status']

grouped_data.head()



In [2]:
columns_to_produce_graphs_for = ['economic_status']
grouped_data = grouped_data.dropna(subset=columns_to_produce_graphs_for)
contacts_columns = ['tot_contacts_wp_wk_nld', 'tot_contacts_wp_wk_ld']

# Create places to store the x and y values of each kde
nld_x_vals = []
ld_x_vals = []

nld_y_vals = []
ld_y_vals = []

category_dict = dict()

for cat_col in columns_to_produce_graphs_for:
    unique_cats = grouped_data[cat_col].unique()
    if cat_col == 'age_5':
        unique_cats = ['15-29 yrs', '30-44 yrs', '45-64 yrs', '65+ yrs']
    if cat_col == 'ses_5':
        unique_cats = [1.0, 2.0, 3.0, 4.0, 5.0]
    # unique_cats = [unique_cats[i] for i in sorted(np.unique(unique_cats, return_index=True)[1])]
    for cat in unique_cats:
        print(cat)
        df = grouped_data.loc[grouped_data[cat_col] == cat]
        # Create the kde line for this category
        fig, ax = plt.subplots()
        line = sns.kdeplot(df['tot_contacts_wp_wk_nld'], label=cat)
        # Get the actual line from the graph
        try:
            line = line.lines[0]
        except IndexError:
            pass
        plt.close(fig)

        fig, ax = plt.subplots()
        line2 = sns.kdeplot(df['tot_contacts_wp_wk_ld'])

        # Get the actual line from the graph
        try:
            line2 = line2.lines[0]
        except IndexError:
            pass

        # Get the corresponding x and y values so we can use them later
        try:
            x_nld, y_nld = line.get_data()
            x_ld, y_ld = line2.get_data()
        except AttributeError:
            pass

        # Store this information in a dictionary
        category_dict[str(cat) + "__nld"] = [x_nld, y_nld]
        category_dict[str(cat) + "__ld"] = [x_ld, y_ld]

        # Store these x and y values
        nld_x_vals.append(x_nld)
        nld_y_vals.append(y_nld)
        ld_x_vals.append(x_ld)
        ld_y_vals.append(y_ld)
        plt.close()


# iterate over the names to plot the ld and nld variants
y_vals_for_scaling = [0]
for col in category_dict.keys():
    if '__nld' in col:
        x, y = category_dict[col]
        y_vals_for_scaling.append(max(y))

# We now need to calculate the cumulative y_maxs values to offset out curves
y_offset = np.cumsum(y_vals_for_scaling)
# We'll likely need to scale the y values some more so we'll do so here
y_offset *= 1.5
colours = ['r', 'b'] * 19
doubled_offset = []
for i in y_offset:
    for j in range(0, 2):
        doubled_offset.append(i)

new_cat_dict = dict()

for idx, (x, y) in enumerate(category_dict.values()):
    new_cat_dict[list(category_dict.keys())[idx]] = [x, y + doubled_offset[idx]]
plt.figure(figsize=(10, 12))
for idx, (x, y) in enumerate(new_cat_dict.values()):
    # First off plot the line, specifying the colour from colours
    plt.plot(x, y, color=colours[idx], alpha=1)
    # Now we need to get a base for the shading, this will just be the minimum value of the y_values for this curve.
    # For the function plt.fill_between, we need to have the same number of x, y_min and y_max values otherwise the
    # function won't work so let's create the minimum values for this shading
    nld_y_fill_min = [min(y)] * len(x)
    # Now plot the shaded region
    plt.fill_between(x, nld_y_fill_min, y, color=colours[idx], alpha=0.1)

# get unique names for lock down and non-lockdown columns
col_names = [i.split("__")[0] for i in list(category_dict.keys())]
col_names = [col_names[i] for i in sorted(np.unique(col_names, return_index=True)[1])]
print(col_names)
new_names = []
for name in col_names:
    name_to_add = ''
    if name == '5.0':
        name_to_add = 'ses_5'
    elif name == '4.0':
        name_to_add = 'ses_4'
    elif name == '3.0':
        name_to_add = 'ses_3'
    elif name == '2.0':
        name_to_add = 'ses_2'
    elif name == '1.0':
        name_to_add = 'ses_1'
    else:
        name_to_add = name
    new_names.append(name_to_add)
plt.subplots_adjust(left=0.3)
orig_xlim = plt.xlim()
orig_ylim = plt.ylim()

plt.plot([-1000, -1000], [-1000, -1100], label='Non-lockdown', color='r', alpha=1)
plt.plot([-1000, -1000], [-1000, -1100], label='Lockdown', color='b', alpha=1)
plt.xlim(orig_xlim)
plt.ylim(orig_ylim)
plt.yticks(y_offset[:-1], labels=new_names, fontsize=10)
plt.xlabel('Number of contacts')
plt.legend(loc='upper right', fancybox=True, framealpha=0.5)
#plt.title("Distribution of weekday contacts across occupations at work/school \nduring lockdown and non-lockdown") #, weight='bold'
plt.savefig(path_output+ "stacked_kde_occ_at_work"+cut_off+".png", dpi=300, facecolor='w')

NameError: name 'grouped_data' is not defined